In [54]:
from bs4 import BeautifulSoup as bs
import lxml
import nltk
import os
import string

nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import pandas as pd

df = pd.read_json("people.json")

import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np
np.random.seed(400)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jason\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
def extract_info(filepath,words):
    
    dictionary_of_interest = {}
    
    with open(filepath,"r",encoding="utf8") as file:
            
        content = file.readlines()
        content = "".join(content)
        
        bs_content = bs(content, "lxml")
        
        unique_id = bs_content.find("tei").attrs["xml:id"]
        
        dictionary_of_interest["unique_id"] = unique_id
        
        letter_details = bs_content.find_all("correspaction")
        
        for deets in letter_details:
            
            if deets.attrs["type"] == "sent":
                
                if "when" in list(deets.date.attrs.keys()):
                    dictionary_of_interest["date"] = deets.date.attrs["when"]
                
                try:
                    dictionary_of_interest["sender"] = deets.persname.text
                    if dictionary_of_interest["sender"] != "Darwin, C. R.":
                        file.close()
                        return "Not Darwin Writing"
                except AttributeError:
                    dictionary_of_interest["reciever"] = deets.orgname.text
                
                try:
                    dictionary_of_interest["sender_bio"] = deets.persname.attrs["key"]
                except AttributeError:
                    dictionary_of_interest["sender_bio"] = "None Available"
                except KeyError:
                    dictionary_of_interest["sender_bio"] = "None Available"
                    
            if deets.attrs["type"] == "received":
                
                try:
                    dictionary_of_interest["reciever"] = deets.persname.text
                except AttributeError:
                    dictionary_of_interest["reciever"] = deets.orgname.text
                    
                try:
                    dictionary_of_interest["reciever_bio"] = deets.persname.attrs["key"]
                except AttributeError:
                    dictionary_of_interest["reciever_bio"] = "None Available"
                except KeyError:
                    dictionary_of_interest["reciever_bio"] = "None Available"
                    
        try:
            free_text = bs_content.find_all("div",{"type":"transcription"})[0].p.text
        except AttributeError:
#             print(bs_content) 
            free_text = ""
            

        # cleaning of the data
        dictionary_of_interest["body"] = (free_text.lower()).translate(str.maketrans('','',string.punctuation))
        text_tokens = word_tokenize(dictionary_of_interest["body"])
#         no_stop_words = [word for word in text_tokens] # if not word in stopwords.words()
        for word in text_tokens:
            words.append(word)
#         print(no_stop_words)!

        file.close()
        
    return dictionary_of_interest

def generate_feature_data(free_text,feature_set):
    
    feature_bools = []
    
    for word in feature_set:
        feature_bools.append(1*(word in free_text))
        
    return feature_bools

In [12]:
path = "dcp-data/letters/"
files = os.listdir(path)
files = files[1:]
words = []
# print(files[1])
i = 0
cap = 0

if cap == 0:
    cap = len(files)
    
data_set = []
for file_target in files:
    dict_test = extract_info(path+file_target,words)
    if dict_test != "Not Darwin Writing":
        data_set.append(dict_test)
#     print(dict_test)
    if i == cap:
        break
    elif i < cap:
        i += 1
    else:
        print("Failed loop")
        break
    print(round((i/cap)*100,2),end="\r"*(i!=cap))

print("")
print("Analysis Finished")
print(f"When all words are extracted, we have got a dataset of {len(words)} words")

100.0
Analysis Finished
When all words are extracted, we have got a dataset of 705858 words


In [13]:

if "dump.txt" not in os.listdir("."):
    unique_words = {}
    for counter, word in enumerate(words):
        try:
            unique_words[word] += 1
        except KeyError:
            unique_words[word] = 1
        print(round(((counter+1)/len(words))*100,2),end="\r")

    sorted_unique_words = {key: value for key, value in sorted(unique_words.items(), key=lambda item: item[1],reverse=True)}

    print(list(sorted_unique_words.keys())[:1000])
    feature_words = list(sorted_unique_words.keys())[:4000]
    with open("dump.txt","w",encoding="utf8") as output:
        for word in feature_words:
            try:
                output.write(word +"\n")
            except:
                print(word)

    output.close()

with open('dump.txt', encoding="utf8") as f:
    lines = f.readlines()
    
lines_cleaned = []
for word in lines:
    word = word[:-1]
    lines_cleaned.append(word)
    
feature_words = lines_cleaned
# print(sum(list(sorted_unique_words.values())[:4000]))
# print(sum(list(sorted_unique_words.values())[4000:]))

In [88]:
test = generate_feature_data(data_set[0]["body"],feature_words)
generate_feature_data("I am a woman, ducks are nice",feature_words)
complete_data = []
targets = []
for dictionary in data_set:
    free_text = dictionary["body"]
    try:
        reciever_id = dictionary["reciever_bio"]
    except:
        print(dictionary)
        continue
    number_key = reciever_id[21:-4]
    
    boolean_set = generate_feature_data(free_text,feature_words)
    try:
        gender = df[df["id"]=="DCP-IDENT-"+str(number_key)]["sex"].iloc[0]
        if gender == "":
            gender = "NotAvailable"
#             print(number_key)
    except:
        continue
#         gender = "NotAvailable"
#         print(number_key)
#         print(reciever_id)
#         print(df[df["id"]=="DCP-IDENT-"+str(number_key)])
        
    complete_data.append(boolean_set)
    targets.append(gender)
#     if counter == 10:
#         break
#     else:
#         counter += 1
# print(dict_test["sender"])
# print(dict_test["reciever"])
# print(test)

7510
9999
9999
9999
7861
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
6538
6573
9999
9999
6573
9999
9999
7475
7475
9999
7475
7475
9999
9999
7475
7831
9999
9999
9999
9999
9999
9999
7803
5454
9999
9999
9999
3570
9999
9999
9999
7831
2134
7299
7299
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
926
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
7868
7868
7868
7868
7868
7868
{'unique_id': 'DCP-LETT-2115F', 'sender': 'Darwin, C. R.', 'sender_bio': '../nameregs/nameregs_1.xml', 'body': '– july\xa01857—'}
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
1202
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
9999
3742
2581
6393
6424
5882
9999
9999
5882
5882
9999
7223
9999
9999
9999
9999
9999
9999
9999
5813
9999
9999
9999
9999
6054
9999
9999
9999
64
9999
9999
9999
9999
9999
7373
6660
9999
5362
9999
662

In [53]:
unique_tags = {}
for val in targets:
    try:
        unique_tags[val] += 1
    except:
        unique_tags[val] = 1
        
print(unique_tags)
print(list(unique_tags.values())[0]/sum(unique_tags.values()))

{'M': 7277, 'NotAvailable': 306, '': 225, 'F': 324}
0.8948598130841121


In [60]:
X_train, X_test, y_train, y_test = train_test_split(
    complete_data, targets, test_size=0.2, random_state=42
)

In [61]:
classifier = KNeighborsClassifier(30)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print(score)

0.8813767670559312


In [73]:
classifier = DecisionTreeClassifier(max_depth=10)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

In [74]:
out = classifier.predict(X_test)
classifier.predict(test)

In [76]:
for i,val in enumerate(out):
    if val != y_test[i]:
        print(val,y_test[i],i)

M NotAvailable 0
M F 1
F M 20
M F 23
M  24
M NotAvailable 47
M  70
M F 90
M  94
M NotAvailable 110
M NotAvailable 111
M NotAvailable 120
M NotAvailable 124
F M 156
M NotAvailable 159
M NotAvailable 164
M F 167
M F 173
M F 178
M  182
M NotAvailable 200
M NotAvailable 209
M F 220
M F 233
NotAvailable M 237
M NotAvailable 241
M NotAvailable 253
M F 257
M NotAvailable 263
M F 275
M F 276
M NotAvailable 280
M NotAvailable 284
M  287
M NotAvailable 288
M  316
M F 317
M  318
M NotAvailable 324
M NotAvailable 328
M  372
M F 382
M F 395
M F 406
M NotAvailable 415
M F 418
F M 434
M F 449
M NotAvailable 454
M  463
M  468
M  477
M NotAvailable 481
 M 512
M NotAvailable 529
M NotAvailable 533
M F 536
M  537
M NotAvailable 540
M F 546
M  547
M F 554
M F 561
M NotAvailable 575
M NotAvailable 577
M NotAvailable 584
M  585
M F 594
M NotAvailable 595
M F 601
M F 604
F M 615
M F 616
F M 621
M F 640
M NotAvailable 654
F  662
M  680
M F 681
M  684
M F 690
M F 694
M  717
M NotAvailable 719
M  721
M F 722
F 

In [87]:
# print(len(classifier.feature_importances_))

test_words = []
counter = 0
for i,val in enumerate(classifier.feature_importances_):
    if val != 0:
        counter += 1
        print(feature_words[i],val)
        
print(counter)

you 0.06098600433356366
if 0.010962624869056447
was 0.016005989277393818
’ 0.013483871100532006
many 0.024996687288673648
see 0.005695353497336623
work 0.010125072884153988
c 0.012492934437014925
first 0.012204328922864193
far 0.01044148141178381
kindness 0.024550558456631028
better 0.005695353497336623
written 0.01079304644248359
nothing 0.00683442419680395
receive 0.01704559944300416
says 0.023555295765375895
father 0.05699173113100981
pay 0.003796902331557749
further 0.008284150541580543
d 0.019194508536196864
st 0.015946989792542547
plan 0.019102427879265683
land 0.008775063166266799
children 0.01451209197348006
set 0.012656341105192497
service 0.013400831758439112
movement 0.00965915263467713
went 0.01066818024192472
erasmus 0.021549603277382638
black 0.00721411442995972
garden 0.029365747714999346
admiration 0.01668597873183174
captain 0.00664457908022606
previous 0.018408057400681674
human 0.014789267135696275
monograph 0.018280371453393736
distance 0.01891513962962701
recommend